In [1]:
import random
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
import joblib
import numpy as np
import pickle

import logging
from Lib.DataLoader import DataLoader
from Lib.Diagnoser import Diagnoser
from Lib.UIEArgs import uie_args

# from paddlenlp import Taskflow
# from pprint import pprint
# from paddlenlp.transformers import AutoTokenizer
import sys
sys.path.append('../../paddlenlp/uie_fitting/deploy/python')

logging.basicConfig(level=logging.ERROR)

from uie_predictor import UIEPredictor

dieases = ["急性阑尾炎", "急性胰腺炎", "肠梗阻", "异位妊娠", "急性胆管炎", "急性胆囊炎", "上尿路结石", "卵巢囊肿", "消化道穿孔"]

sl_diag1 = Diagnoser(r'output/models/diagnose/scaler_gbdt_临床.pkl', r'output/models/diagnose/gbdt_临床_0.6936.m')
sl_diag2 = Diagnoser(r'output/models/diagnose/scaler_gbdt_全特征.pkl', r'output/models/diagnose/gbdt_全特征_0.8633.m')
sl_diag3 = Diagnoser(r'output/models/diagnose/scaler_gbdt_合成.pkl', r'output/models/diagnose/gbdt_合成_0.8386.m')
ml_diag = UIEPredictor(uie_args(model_path_prefix=r'model_nano/inference',
  schema='预测疾病[急性阑尾炎,异位妊娠,急性胆囊炎,急性胰腺炎,上尿路结石,卵巢囊肿,肠梗阻,消化道穿孔,急性胆管炎]',
                                            device_id = 1))


D:\Anaconda3\lib\site-packages\paddlenlp\transformers\image_utils.py:213: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  resample=Image.BILINEAR,
D:\Anaconda3\lib\site-packages\paddlenlp\transformers\image_utils.py:379: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST,
D:\Anaconda3\lib\site-packages\paddlenlp\transformers\ernie_vil\feature_extraction.py:65: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resample=Image.BICUBIC,
D:\Anaconda3\lib\site-packages\paddlenlp\transformers\clip\feature_extraction.py:64: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resample=Image.BICUBIC,
[2022-11-29 10:27:14,250] [    INFO] - We are using <class 'paddlenlp.t

>>> [InferBackend] Creating Engine ...
>>> [InferBackend] Use CPU to inference ...
>>> [InferBackend] Engine Created ...


In [2]:
def load_multilbl_data(file_path, num_fields, separator, skip_title, shuffle=True):
    dl = DataLoader()
    lines = dl.load_data_lines(file_path, num_fields=num_fields, separator=separator, skip_title=skip_title, shuffle=shuffle)
    X = [[int(e) for e in l[11:]] for l in lines]
    y = [[int(e) for e in l[2:11]] for l in lines]
    ids = [l[0:2] for l in lines]

    return X, y, ids

In [13]:
num_fields = 184
# X_test, y_test, ids_test = load_multilbl_data(r'data/uie/测试_全特征_多诊断.txt', \
#                                               num_fields=num_fields, separator='	', skip_title=True, shuffle=False)
X_test, y_test, ids_test = load_multilbl_data(r'data/uie/测试数据_合成_多诊断.txt', \
                                              num_fields=num_fields, separator='	', skip_title=True, shuffle=False)

print(X_test[0])
print(y_test[0])
print(ids_test[0])

[0, 1, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 58, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 1, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 0, 0, 1, 1, 0, 0, 0]
['10244270', '20211220']


In [4]:
# def merge_results(r1_dict, r2_dict, r1_delta):
#     """
#     合并单诊断和多诊断的结果
#     """
#     r1_set = {r for r in r1_dict}
#     r2_set = {r for r in r2_dict}
    
#     result = []
#     if len(r1_set) == 0:
#         for e in r2_set:
#             if r2_dict[e] >= 0.9:
#                 result.append(e)
#         if len(result) == 0:
#             result = list(r2_set)
#     elif len(r2_set) == 0:
#         result = list(r1_set)
#     else:
#         r1_text = list(r1_set)
#         has_high_score = False
#         for t in r1_text:
#             if r1_dict[t] >= r1_delta:
#                 has_high_score = True
#                 break
        
#         if has_high_score:
#             result = r1_text
#         else:
#             r12_set = r1_set.intersection(r2_set)
#             result = list(r12_set)
#             for e in r1_set - r12_set:
#                 if r1_dict[e] >= 0.5:
#                     result.append(e)
                    
#             for e in r2_set - r12_set:
#                 if r2_dict[e] >= 0.9:
#                     result.append(e)
# #     print(result)
#     return result


In [27]:
def merge_results(dict_arr):
    result_dict = {}
    for dct in dict_arr:
        for k, v in dct.items():
            if k not in result_dict:
                result_dict[k] = {'scores': [], 'ct': 0}
            result_dict[k]['scores'].append(v)
            result_dict[k]['ct'] += 1
            
    result_arr = [(k, v['ct'], np.mean(v['scores']), np.max(v['scores'])) for k, v in result_dict.items()]
    result_arr = sorted(result_arr, key=lambda x: (-x[1], -x[2]))
    print(result_arr)
    result = []
    
    if len(dict_arr) == 4:
        for r in result_arr:
            if r[1] >= 2:
                result.append(r[0])
    elif len(dict_arr) == 3:
        for r in result_arr:
            if r[1] >= 2:
                result.append(r[0])
    elif len(dict_arr) == 2:
        for r in result_arr:
            result.append(r[0])
            break
    return result

In [31]:
def text_result_to_labels(results):
    """
    文本预测结果转换为多标签标注
    """
    results_ = []
    for r in results:
        r_ = [0 for i in range(len(dieases))]
        for t in r:
            r_[dieases.index(t)] = 1
        results_.append(r_)
        
    return results_

def labels_to_dieases(l):
    result = []
    for idx, r in enumerate(l):
        if r == 1:
            result.append(dieases[idx])
            
    return '，'.join(result)
    

def union_pred(X_test, y_test, r1_delta):
    """
    联合预测
    """
    results = []
    for idx, (X, y, ids) in enumerate(zip(X_test, y_test, ids_test)):
#         if idx not in [7,28,32,59,67,71,125]: #[7,8,17,27,28,32,33,36,59,67,71,81,83,103,112,113,125]:
#             continue
            
#         print('')
        print(idx + 2)
        # 单诊断
        result_sl1 = {}
        pred = sl_diag1.predict_batch([X], pred_num=None, prob_delta=0.5, out_format='dict')[0]
        for r1 in pred:
            result_sl1[r1['name']] = r1['prob']
        print(result_sl1)
        
        result_sl2 = {}
        pred = sl_diag2.predict_batch([X], pred_num=None, prob_delta=0.5, out_format='dict')[0]
        for r1 in pred:
            result_sl2[r1['name']] = r1['prob']
        print(result_sl2)
        
        result_sl3 = {}
        pred = sl_diag3.predict_batch([X], pred_num=None, prob_delta=0.5, out_format='dict')[0]
        for r1 in pred:
            result_sl3[r1['name']] = r1['prob']
        print(result_sl3)
        

        # 多诊断
        result_ml = {}
        text = '，'.join([str(x) for x in X])
        for r_item in ml_diag.predict([text]):
            for k, v in r_item.items():
                for v_item in v:
                    result_ml[v_item['text']] = v_item['probability']
                
        print(result_ml)
#         result = merge_results([result_sl1] + [result_sl2] + [result_sl3] + [result_ml])
        result = merge_results([result_sl3] + [result_ml])
        print(result)
        print(labels_to_dieases(y))
        
        results.append(result)

    return results

def single_pred(X_test):
    results = []
    for idx, (X, y, ids) in enumerate(zip(X_test, y_test, ids_test)):
        # 单诊断
        pred = sl_diag3.predict_batch([X], pred_num=None, prob_delta=0.5, out_format='dict')[0]
        results.append([r1['name'] for r1 in pred])

    return results

def calc_score(results, y_test):
    accuracy = accuracy_score(y_test, results)
    f1_score_micro = f1_score(y_test, results, average='micro')
    f1_score_macro = f1_score(y_test, results, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

In [32]:
results1 = text_result_to_labels(union_pred(X_test, y_test, 0.5))
calc_score(results1, y_test)

2
{'急性胆囊炎': 0.965697037622785}
{'急性胆管炎': 0.9960993277605129}
{'急性胆管炎': 0.99996104868217}
{'急性胆囊炎': 0.9508447110238194}
[('急性胆管炎', 1, 0.99996104868217, 0.99996104868217), ('急性胆囊炎', 1, 0.9508447110238194, 0.9508447110238194)]
['急性胆管炎']
急性胆管炎，急性胆囊炎
3
{'急性胰腺炎': 0.9965829755145647}
{'肠梗阻': 0.9999998459344231}
{'肠梗阻': 0.9999997844734595}
{'肠梗阻': 0.9927311057943768}
[('肠梗阻', 2, 0.9963654451339181, 0.9999997844734595)]
['肠梗阻']
肠梗阻
4
{'急性胆管炎': 0.9997404359249045}
{'急性胆管炎': 0.9999999988010995}
{'急性胆管炎': 0.9999999713779766}
{'急性胆管炎': 0.9885779618173984}
[('急性胆管炎', 2, 0.9942889665976875, 0.9999999713779766)]
['急性胆管炎']
急性胆管炎
5
{'急性胆管炎': 0.9998966809506646}
{'急性胆管炎': 0.9999999663411524}
{'急性胆管炎': 0.9999997605005995}
{'急性胆管炎': 0.9741205519520122}
[('急性胆管炎', 2, 0.9870601562263058, 0.9999997605005995)]
['急性胆管炎']
急性胆管炎
6
{'急性胆管炎': 0.9970108451836724}
{'急性胆管炎': 0.9999999960912724}
{'急性胆管炎': 0.9999996964045151}
{'急性胆管炎': 0.9857852005817556}
[('急性胆管炎', 2, 0.9928924484931354, 0.9999996964045151)]
['急性胆管炎']


{'急性胰腺炎': 0.9999981642898845}
{'急性胰腺炎': 0.9999999921288456}
{'急性胰腺炎': 0.9999999195217933}
{'急性胰腺炎': 0.9979025405219062}
[('急性胰腺炎', 2, 0.9989512300218497, 0.9999999195217933)]
['急性胰腺炎']
急性胰腺炎
46
{'上尿路结石': 0.9999999937404382}
{'上尿路结石': 0.9999921812248341}
{'上尿路结石': 0.9999999854933463}
{'上尿路结石': 0.9937855408222163}
[('上尿路结石', 2, 0.9968927631577813, 0.9999999854933463)]
['上尿路结石']
上尿路结石
47
{'上尿路结石': 0.9999999800422821}
{'上尿路结石': 0.9999997518825797}
{'上尿路结石': 0.9999999931893271}
{'上尿路结石': 0.9886229673652025}
[('上尿路结石', 2, 0.9943114802772648, 0.9999999931893271)]
['上尿路结石']
上尿路结石
48
{'上尿路结石': 0.9999999901768364}
{'上尿路结石': 0.9999999950233374}
{'上尿路结石': 0.9999999948063341}
{'上尿路结石': 0.999747471029643}
[('上尿路结石', 2, 0.9998737329179885, 0.9999999948063341)]
['上尿路结石']
上尿路结石
49
{'急性阑尾炎': 0.9999999995219646}
{'急性阑尾炎': 0.9999999940618256}
{'急性阑尾炎': 0.9999999964501995}
{'急性阑尾炎': 0.9995388907196201}
[('急性阑尾炎', 2, 0.9997694435849098, 0.9999999964501995)]
['急性阑尾炎']
急性阑尾炎
50
{'异位妊娠': 0.9999828656753295}
{'

{'肠梗阻': 0.9928085871413117}
[('肠梗阻', 2, 0.9964041992581576, 0.9999998113750035)]
['肠梗阻']
肠梗阻
89
{'肠梗阻': 0.9086998910337607}
{'肠梗阻': 0.8356596776611247}
{'肠梗阻': 0.5657646446089694}
{'卵巢囊肿': 0.5900870123744539}
[('卵巢囊肿', 1, 0.5900870123744539, 0.5900870123744539), ('肠梗阻', 1, 0.5657646446089694, 0.5657646446089694)]
['卵巢囊肿']
肠梗阻
90
{'上尿路结石': 0.506221009465043}
{'急性阑尾炎': 0.999955116511469}
{'急性阑尾炎': 0.9999976928315212}
{'急性阑尾炎': 0.9550113208956503}
[('急性阑尾炎', 2, 0.9775045068635857, 0.9999976928315212)]
['急性阑尾炎']
急性阑尾炎
91
{'肠梗阻': 0.9999999996023377}
{'肠梗阻': 0.9999973446295759}
{'肠梗阻': 0.999999981458185}
{'肠梗阻': 0.9910528447968545}
[('肠梗阻', 2, 0.9955264131275198, 0.999999981458185)]
['肠梗阻']
肠梗阻
92
{'急性阑尾炎': 0.8430461040757511}
{'急性阑尾炎': 0.9986532602154188}
{'急性阑尾炎': 0.9929933136184296}
{'急性阑尾炎': 0.39951591880918613}
[('急性阑尾炎', 2, 0.6962546162138079, 0.9929933136184296)]
['急性阑尾炎']
急性阑尾炎，肠梗阻
93
{'急性胰腺炎': 0.8960149560157168}
{'急性胰腺炎': 0.9999999911311814}
{'急性胰腺炎': 0.9999996252911094}
{'急性胆囊炎': 

{'急性阑尾炎': 0.9999999982109227}
{'急性阑尾炎': 0.9999999962713595}
{'急性阑尾炎': 0.9998883037422672}
[('急性阑尾炎', 2, 0.9999441500068134, 0.9999999962713595)]
['急性阑尾炎']
急性阑尾炎
133
{'急性胆囊炎': 0.9983229086033631}
{'急性胰腺炎': 0.9999999707676506}
{'急性胰腺炎': 0.9999999759948984}
{'急性胰腺炎': 0.9987136404436825}
[('急性胰腺炎', 2, 0.9993568082192905, 0.9999999759948984)]
['急性胰腺炎']
急性胰腺炎
134
{'急性胆管炎': 0.6643362650387067}
{'上尿路结石': 0.9783081405924399}
{'上尿路结石': 0.9988665236013521}
{'急性胆囊炎': 0.36513354057154856}
[('上尿路结石', 1, 0.9988665236013521, 0.9988665236013521), ('急性胆囊炎', 1, 0.36513354057154856, 0.36513354057154856)]
['上尿路结石']
急性胰腺炎
135
{'急性胆管炎': 0.945292207760929}
{'急性胆管炎': 0.9222323476578771}
{'肠梗阻': 0.5911425178440134}
{'急性胆囊炎': 0.9900289040391286}
[('急性胆囊炎', 1, 0.9900289040391286, 0.9900289040391286), ('肠梗阻', 1, 0.5911425178440134, 0.5911425178440134)]
['急性胆囊炎']
急性胆囊炎
136
{'异位妊娠': 0.9999999998825917}
{'异位妊娠': 0.99999999986583}
{'异位妊娠': 0.9999999855262889}
{'异位妊娠': 0.9992926775697697}
[('异位妊娠', 2, 0.999646331548029

{'急性胆囊炎': 0.9721122068001283}
{'急性胆囊炎': 0.9997932930475472}
{'急性胆囊炎': 0.6264173104141776}
{'急性胆囊炎': 0.886563726677764}
[('急性胆囊炎', 2, 0.7564905185459707, 0.886563726677764)]
['急性胆囊炎']
急性胆囊炎
177
{'急性胰腺炎': 0.9999997809040355}
{'急性胆管炎': 0.9986418660666314}
{'急性胰腺炎': 0.9913355548792911}
{'急性胆管炎': 0.9215474469027924}
[('急性胰腺炎', 1, 0.9913355548792911, 0.9913355548792911), ('急性胆管炎', 1, 0.9215474469027924, 0.9215474469027924)]
['急性胰腺炎']
急性胰腺炎
178
{'肠梗阻': 0.9999999780236817}
{'肠梗阻': 0.9999999968028526}
{'肠梗阻': 0.9999999973588913}
{'肠梗阻': 0.998982504583239}
[('肠梗阻', 2, 0.9994912509710652, 0.9999999973588913)]
['肠梗阻']
肠梗阻
179
{'卵巢囊肿': 0.9996079012694294}
{'卵巢囊肿': 0.9999253191045813}
{'卵巢囊肿': 0.9999479759170721}
{'急性阑尾炎': 0.646005350897326}
[('卵巢囊肿', 1, 0.9999479759170721, 0.9999479759170721), ('急性阑尾炎', 1, 0.646005350897326, 0.646005350897326)]
['卵巢囊肿']
卵巢囊肿
180
{'肠梗阻': 0.9999873819414848}
{'急性阑尾炎': 0.9999931132002986}
{'急性阑尾炎': 0.9999847067577167}
{'急性阑尾炎': 0.9722718327730604}
[('急性阑尾炎', 2, 0.9861

...

{'异位妊娠': 0.978240032814504}
{'异位妊娠': 0.9999919676065655}
{'异位妊娠': 0.9999941354934283}
{'异位妊娠': 0.9985463446342848}
[('异位妊娠', 2, 0.9992702400638566, 0.9999941354934283)]
['异位妊娠']
异位妊娠
2245
{'肠梗阻': 0.9957693098017502}
{'肠梗阻': 0.9997404988407546}
{'肠梗阻': 0.9995468871836736}
{'肠梗阻': 0.9871494244845849}
[('肠梗阻', 2, 0.9933481558341293, 0.9995468871836736)]
['肠梗阻']
肠梗阻
2246
{'卵巢囊肿': 0.9689485290173316}
{'异位妊娠': 0.9994758306469071}
{'异位妊娠': 0.9999959731218878}
{'异位妊娠': 0.9992264807366524}
[('异位妊娠', 2, 0.9996112269292701, 0.9999959731218878)]
['异位妊娠']
异位妊娠
2247
{'急性胆管炎': 0.9226115718472715}
{'急性胆管炎': 0.8948692095262571}
{'急性胰腺炎': 0.9998271417995438}
{'急性胰腺炎': 0.9594994819838902}
[('急性胰腺炎', 2, 0.9796633118917171, 0.9998271417995438)]
['急性胰腺炎']
急性胰腺炎，急性胆管炎，急性胆囊炎
2248
{'上尿路结石': 0.9987712932100717}
{'上尿路结石': 0.9999997693269176}
{'上尿路结石': 0.999999799176124}
{'上尿路结石': 0.9682629263962212}
[('上尿路结石', 2, 0.9841313627861726, 0.999999799176124)]
['上尿路结石']
上尿路结石
2249
{'急性阑尾炎': 0.9999979018116759}
{'急性阑尾炎':

{'急性胰腺炎': 0.9999881332809906}
{'急性胰腺炎': 0.9999999830063652}
{'急性胰腺炎': 0.9995478844107595}
[('急性胰腺炎', 2, 0.9997739337085623, 0.9999999830063652)]
['急性胰腺炎']
急性胰腺炎
2287
{'急性胰腺炎': 0.9715053725862433}
{'上尿路结石': 0.9999997891436454}
{'上尿路结石': 0.9999960981141418}
{'上尿路结石': 0.9266433097020581}
[('上尿路结石', 2, 0.9633197039081, 0.9999960981141418)]
['上尿路结石']
上尿路结石
2288
{'异位妊娠': 0.99999999995363}
{'异位妊娠': 0.9999999995619397}
{'异位妊娠': 0.9999999865602867}
{'异位妊娠': 0.9992858254181414}
[('异位妊娠', 2, 0.999642905989214, 0.9999999865602867)]
['异位妊娠']
异位妊娠
2289
{'上尿路结石': 0.9999977046202115}
{'上尿路结石': 0.99999894420491}
{'上尿路结石': 0.9999972065881051}
{'上尿路结石': 0.9957342365367339}
[('上尿路结石', 2, 0.9978657215624195, 0.9999972065881051)]
['上尿路结石']
上尿路结石
2290
{'异位妊娠': 0.9112354926152203}
{'异位妊娠': 0.9999999995208881}
{'异位妊娠': 0.9999998738882558}
{'异位妊娠': 0.9992497183044691}
[('异位妊娠', 2, 0.9996247960963625, 0.9999998738882558)]
['异位妊娠']
异位妊娠
2291
{'肠梗阻': 0.9999999849734388}
{'肠梗阻': 0.9999999668745782}
{'肠梗阻': 0.999999

{'急性胆囊炎': 0.8997789566879817}
[('急性胆管炎', 1, 0.996781158188175, 0.996781158188175), ('急性胆囊炎', 1, 0.8997789566879817, 0.8997789566879817)]
['急性胆管炎']
急性胆管炎，急性胆囊炎
2328
{'上尿路结石': 0.9984364685993746}
{'急性胆管炎': 0.9995313425362319}
{'急性胆囊炎': 0.5398445345525527}
{'急性胰腺炎': 0.8480354193006292}
[('急性胰腺炎', 1, 0.8480354193006292, 0.8480354193006292), ('急性胆囊炎', 1, 0.5398445345525527, 0.5398445345525527)]
['急性胰腺炎']
急性胰腺炎，急性胆管炎，急性胆囊炎
2329
{'肠梗阻': 0.9968830474030347}
{'肠梗阻': 0.9528308554082382}
{'肠梗阻': 0.9999678464470224}
{'肠梗阻': 0.9415191484739438}
[('肠梗阻', 2, 0.9707434974604832, 0.9999678464470224)]
['肠梗阻']
肠梗阻，消化道穿孔
2330
{'上尿路结石': 0.9972552088074305}
{'上尿路结石': 0.7433293172326834}
{'上尿路结石': 0.8431243844964396}
{'急性阑尾炎': 0.711721932957154}
[('上尿路结石', 1, 0.8431243844964396, 0.8431243844964396), ('急性阑尾炎', 1, 0.711721932957154, 0.711721932957154)]
['上尿路结石']
上尿路结石
2331
{'急性胆管炎': 0.9988479820903102}
{'急性胆管炎': 0.5955272378084551}
{'急性胆管炎': 0.9039287511461113}
{'上尿路结石': 0.9480629735838875}
[('上尿路结石', 1, 0.948

{'异位妊娠': 0.9992588938854396}
[('异位妊娠', 2, 0.9996285710315258, 0.9999982481776121)]
['异位妊娠']
异位妊娠
2369
{'急性胆囊炎': 0.9978955866731642}
{'急性胰腺炎': 0.9999998086092342}
{'急性胰腺炎': 0.9999999502543785}
{'急性胰腺炎': 0.9993109080803109}
[('急性胰腺炎', 2, 0.9996554291673447, 0.9999999502543785)]
['急性胰腺炎']
急性胰腺炎
2370
{'异位妊娠': 0.9999999999654641}
{'异位妊娠': 0.9999999983048156}
{'异位妊娠': 0.9999999855230541}
{'异位妊娠': 0.9991834042196857}
[('异位妊娠', 2, 0.9995916948713699, 0.9999999855230541)]
['异位妊娠']
异位妊娠
2371
{'急性胰腺炎': 0.9999999836108328}
{'急性胰腺炎': 0.8140717078401656}
{'急性胰腺炎': 0.9948274703084022}
{'急性胰腺炎': 0.37434475335110307}
[('急性胰腺炎', 2, 0.6845861118297527, 0.9948274703084022)]
['急性胰腺炎']
急性胰腺炎
2372
{'急性胆管炎': 0.9856856009039255}
{'急性胆管炎': 0.9999337920552569}
{'急性胆管炎': 0.7833124133701411}
{'急性胆囊炎': 0.7459246296799193}
[('急性胆管炎', 1, 0.7833124133701411, 0.7833124133701411), ('急性胆囊炎', 1, 0.7459246296799193, 0.7459246296799193)]
['急性胆管炎']
急性胆管炎
2373
{'急性胆管炎': 0.9957659731052374}
{'急性胆管炎': 0.9999382134260303}
{'急性胆管

{'急性胆管炎': 0.9698605815410536}
{'急性胰腺炎': 0.9997140850368625}
{'急性胰腺炎': 0.9991211267082041}
{'急性胰腺炎': 0.9400074677535741}
[('急性胰腺炎', 2, 0.9695642972308891, 0.9991211267082041)]
['急性胰腺炎']
急性胰腺炎，急性胆囊炎
2410
{'急性胰腺炎': 0.9783731979544718}
{'急性胰腺炎': 0.9999999853240258}
{'急性胰腺炎': 0.9999999782602009}
{'急性胰腺炎': 0.9985645056045769}
[('急性胰腺炎', 2, 0.9992822419323889, 0.9999999782602009)]
['急性胰腺炎']
急性胰腺炎
2411
{'急性胰腺炎': 0.7849976800073596}
{'急性胰腺炎': 0.5249713810610135}
{'急性胰腺炎': 0.8561543595865493}
{'急性胰腺炎': 0.44506954805910937}
[('急性胰腺炎', 2, 0.6506119538228293, 0.8561543595865493)]
['急性胰腺炎']
肠梗阻
2412
{'急性胆管炎': 0.9989961450134093}
{'急性胆管炎': 0.9999263675709865}
{'急性胆管炎': 0.9999767442077587}
{'急性胆管炎': 0.8691872369648621}
[('急性胆管炎', 2, 0.9345819905863104, 0.9999767442077587)]
['急性胆管炎']
急性胆管炎
2413
{'肠梗阻': 0.9999999771593668}
{'肠梗阻': 0.9999999020238993}
{'肠梗阻': 0.9999998366601215}
{'肠梗阻': 0.9988478350053143}
[('肠梗阻', 2, 0.9994238358327179, 0.9999998366601215)]
['肠梗阻']
肠梗阻
2414
{'急性胰腺炎': 0.9378106246995841}

In [19]:
results1 = text_result_to_labels(single_pred(X_test))
calc_score(results1, y_test)

Accuracy Score = 0.7742998352553542
F1 Score (Micro) = 0.821592649310873
F1 Score (Macro) = 0.7845213160447488


In [40]:
def merge_results(dict_arr):
    result_dict = {}
    for dct in dict_arr:
        name = dct['name']
        if name not in result_dict:
            result_dict[name] = {'scores': [], 'ct': 0}
        result_dict[name]['scores'].append(dct['prob'])
        result_dict[name]['ct'] += 1

    result_arr = [(k, v['ct'], np.mean(v['scores']), np.max(v['scores'])) for k, v in result_dict.items()]
    result_arr = sorted(result_arr, key=lambda x: (-x[1], -x[2]))
    result = []

    for idx, r in enumerate(result_arr):
        if r[1] >= 2 or idx == 0:
            result.append((r[0], r[3]))
    return result


def union_pred(features, r1_delta=0.5):
    """
    单诊断和多诊断联合预测
    """
    # 单诊断
    pred1 = sl_diag1.predict_batch([features], pred_num=1, prob_delta=0.5, out_format='dict')[0]
    pred2 = sl_diag2.predict_batch([features], pred_num=1, prob_delta=0.5, out_format='dict')[0]
    pred3 = sl_diag3.predict_batch([features], pred_num=1, prob_delta=0.5, out_format='dict')[0]

    # 多诊断
    result_ml = []
    text = '，'.join([str(x) for x in features])
    # {'预测疾病[急性阑尾炎,异位妊娠,急性胆囊炎,急性胰腺炎,上尿路结石,卵巢囊肿,肠梗阻,消化道穿孔,急性胆管炎]': [{'probability': 0.580092666842333, 'text': '上尿路结石'}]}
    for r_item in ml_diag.predict([text]):
        for k, v in r_item.items():
            for v_item in v:
                result_ml.append({'name': v_item['text'], 'prob': v_item['probability']})

    result = []
    for name, prob in merge_results(pred1 + pred2 + pred3 + result_ml):
        result.append({'name': name, 'confirm': 0, 'prob': prob})

    return result

results = []
for idx, (X, y, ids) in enumerate(zip(X_test, y_test, ids_test)):
    result = []
    for r in union_pred(X):
        result.append(r['name'])
    results.append(result)
results1 = text_result_to_labels(results)
calc_score(results1, y_test)

[{'name': '急性胆囊炎', 'confirm': 0, 'prob': 0.965697037622785}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9960993277605129}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.99996104868217}, {'name': '急性胆囊炎', 'prob': 0.9508447110238194}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9965829755145647}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999998459344231}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999997844734595}, {'name': '肠梗阻', 'prob': 0.9927311057943768}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9997404359249045}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999999988010995}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999999713779766}, {'name': '急性胆管炎', 'prob': 0.9885779618173984}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9998966809506646}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999999663411524}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999997605005995}, {'name': '急性胆管炎', 'prob': 0.9741205519520122}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9970108451836724}, {'name': '急性胆管炎', 'conf

[{'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9991692554698922}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9997338885107082}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999930067583089}, {'name': '肠梗阻', 'prob': 0.9986348840608343}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999771037059495}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999820116516326}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999979241782346}, {'name': '急性胆管炎', 'prob': 0.7044946926110391}]
[{'name': '上尿路结石', 'confirm': 0, 'prob': 0.9823135770752522}, {'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.999999995655882}, {'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9999999169030377}, {'name': '急性阑尾炎', 'prob': 0.9997923478914572}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999962929363699}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.999999876718507}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999996155521604}, {'name': '急性胆囊炎', 'prob': 0.8224450113159634}]
[{'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9999999993148236}, {'name': '急性阑尾炎', 'con

[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.5288320446984005}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999638505865178}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9998912754855828}, {'name': '急性胆管炎', 'prob': 0.7697704263093783}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999877945136}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999973282048}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999946862435}, {'name': '急性胰腺炎', 'prob': 0.9986967321785869}]
[{'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999999990946176}, {'name': '上尿路结石', 'confirm': 0, 'prob': 0.7954524501612844}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.999999991010629}, {'name': '肠梗阻', 'prob': 0.9732514031221697}]
[{'name': '肠梗阻', 'confirm': 0, 'prob': 0.9870245604064753}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999997322594755}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999999854858412}, {'name': '肠梗阻', 'prob': 0.9871314701545941}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.8951448783946425}, {'name': '急性胆管炎', 'confirm': 

[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.997297499105671}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9983696399665174}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.7992330456708459}, {'name': '急性胆囊炎', 'prob': 0.9021446333083531}]
[{'name': '肠梗阻', 'confirm': 0, 'prob': 0.8823571059252797}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9967647094597204}, {'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.8076393600109031}, {'name': '急性阑尾炎', 'prob': 0.9668628490708464}]
[{'name': '上尿路结石', 'confirm': 0, 'prob': 0.9889490671952503}, {'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9999999995680895}, {'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9999999964952764}, {'name': '急性阑尾炎', 'prob': 0.9998360936866391}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999266174449}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999983137968}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999970130649}, {'name': '急性胰腺炎', 'prob': 0.9952287876203272}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999442835603}, {'name': '急性胰腺炎', '

[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.999999887538552}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999941073959777}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999999816340408}, {'name': '急性胆管炎', 'prob': 0.9215699548077083}]
[{'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9787794015469444}, {'name': '消化道穿孔', 'confirm': 0, 'prob': 0.999881134144533}, {'name': '消化道穿孔', 'confirm': 0, 'prob': 0.9999073580136685}, {'name': '消化道穿孔', 'prob': 0.9971451822767108}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9997485571406133}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999959309971}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999903671064}, {'name': '急性胆管炎', 'prob': 0.3851350789986441}]
[{'name': '上尿路结石', 'confirm': 0, 'prob': 0.9780084304627188}, {'name': '上尿路结石', 'confirm': 0, 'prob': 0.9999990701926007}, {'name': '上尿路结石', 'confirm': 0, 'prob': 0.9999999592815277}, {'name': '上尿路结石', 'prob': 0.700503659149156}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999828055201717}, {'name': '急性胆管炎',

...

[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.885030939111339}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.993684736527156}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.8716890428177114}, {'name': '急性胆管炎', 'prob': 0.6834697698402223}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9997838599867622}, {'name': '急性胆囊炎', 'confirm': 0, 'prob': 0.7178776898630693}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9510917495715706}, {'name': '急性胆囊炎', 'prob': 0.9820621890304437}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999998626630854}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999993712439879}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999285897488}, {'name': '急性胰腺炎', 'prob': 0.9992134316050851}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9945482447458808}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999998992791086}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999894278062}, {'name': '急性胰腺炎', 'prob': 0.9889081590570825}]
[{'name': '肠梗阻', 'confirm': 0, 'prob': 0.659593089580048}, {'name': '异位妊娠', 'c

[{'name': '肠梗阻', 'confirm': 0, 'prob': 0.9810583706039352}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9997051735436601}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9997808498202002}, {'name': '上尿路结石', 'prob': 0.3297029803915663}]
[{'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9991692554698922}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9995794684361611}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999928894718076}, {'name': '肠梗阻', 'prob': 0.9986323854729307}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.7984791948295884}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9014004646153061}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9806431979984769}, {'name': '急性胆囊炎', 'prob': 0.9628207321414095}]
[{'name': '肠梗阻', 'confirm': 0, 'prob': 0.9993018837881381}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999420044483592}, {'name': '肠梗阻', 'confirm': 0, 'prob': 0.9999999886242676}, {'name': '肠梗阻', 'prob': 0.9922031414473338}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9783731979544718}, {'name': '急性胰腺炎', 'confirm': 0, 'p

[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9000100657246584}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9992093463042352}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9997492033345475}, {'name': '急性胆管炎', 'prob': 0.9869187136808364}]
[{'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9999501083833201}, {'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.999999998266139}, {'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9999999963165305}, {'name': '急性阑尾炎', 'prob': 0.999884131603082}]
[{'name': '异位妊娠', 'confirm': 0, 'prob': 0.9999975849723389}, {'name': '异位妊娠', 'confirm': 0, 'prob': 0.9999992044148133}, {'name': '异位妊娠', 'confirm': 0, 'prob': 0.9999994635487273}, {'name': '异位妊娠', 'prob': 0.9913213755632455}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.8951448783946425}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.901288271464981}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9882937678922784}, {'name': '上尿路结石', 'prob': 0.7885077634365842}]
[{'name': '急性阑尾炎', 'confirm': 0, 'prob': 0.9999999977002823}, {'name': '急性阑尾炎', 'co

[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999836108328}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.8140717078401656}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9948274703084022}, {'name': '急性胰腺炎', 'prob': 0.37434475335110307}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9856856009039255}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999337920552569}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.7833124133701411}, {'name': '急性胆囊炎', 'prob': 0.7459246296799193}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9957659731052374}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999382134260303}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999924901559434}, {'name': '急性胆管炎', 'prob': 0.7405326719646865}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9897855323342748}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999396583021}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999895726237}, {'name': '急性胰腺炎', 'prob': 0.9993379615241693}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9970108451836724}, {'name': '急性胆

[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9989961450134093}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999263675709865}, {'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9999767442077587}, {'name': '急性胆管炎', 'prob': 0.8691872369648621}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9698605815410536}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9997140850368625}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9991211267082041}, {'name': '急性胰腺炎', 'prob': 0.9400074677535741}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9783731979544718}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999853240258}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.9999999782602009}, {'name': '急性胰腺炎', 'prob': 0.9985645056045769}]
[{'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.7849976800073596}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.5249713810610135}, {'name': '急性胰腺炎', 'confirm': 0, 'prob': 0.8561543595865493}, {'name': '急性胰腺炎', 'prob': 0.44506954805910937}]
[{'name': '急性胆管炎', 'confirm': 0, 'prob': 0.9989961450134093}, {'name': '急性胆

In [22]:
results1 = text_result_to_labels(union_pred(X_test, y_test, 0.5))
calc_score(results1, y_test)

2
{'肠梗阻': 0.9999999659165436}
{'肠梗阻': 0.999717652797699}
['肠梗阻']
['肠梗阻']
3
{'消化道穿孔': 0.9039818226991675}
{'消化道穿孔': 0.979031503200531}
['消化道穿孔']
['消化道穿孔']
4
{'上尿路结石': 0.9999990173352235}
{'上尿路结石': 0.9762212038040161}
['上尿路结石']
['上尿路结石']
5
{'肠梗阻': 0.9999987672145347}
{'肠梗阻': 0.9988495111465454}
['肠梗阻']
['肠梗阻']
6
{'急性胰腺炎': 0.9999996776973803}
{'急性胆囊炎': 0.9421985149383545, '急性胰腺炎': 0.9984879493713379}
['急性胰腺炎']
['急性胰腺炎']
7
{'急性胆管炎': 0.9999943033888808}
{'急性胆囊炎': 0.9802579283714294, '急性胆管炎': 0.8511038422584534}
['急性胆管炎']
['急性胆管炎']
8
{'上尿路结石': 0.9999900399410319}
{'上尿路结石': 0.9863721132278442}
['上尿路结石']
['上尿路结石']
9
{'卵巢囊肿': 0.5681908988861422}
{'急性阑尾炎': 0.9772273302078247}
['卵巢囊肿']
['卵巢囊肿']
10
{'急性胰腺炎': 0.9799711337775067}
{'急性胰腺炎': 0.9751476049423218, '急性胆囊炎': 0.9438185095787048, '急性胆管炎': 0.8964278697967529}
['急性胰腺炎']
['急性胆管炎']
11
{'急性胰腺炎': 0.9999999995637072}
{'急性胰腺炎': 0.999416708946228}
['急性胰腺炎']
['急性胰腺炎']
12
{'急性阑尾炎': 0.9951487959850579}
{'急性阑尾炎': 0.9989224076271057}
['急性阑尾炎']
['卵巢囊肿']
13

{'上尿路结石': 0.9219354391098022, '急性胆囊炎': 0.9939970970153809}
['急性胆囊炎']
['急性胆囊炎']
90
{'上尿路结石': 0.9999999791978385}
{'上尿路结石': 0.9900058507919312}
['上尿路结石']
['上尿路结石']
91
{'急性阑尾炎': 0.9999999998079065}
{'急性阑尾炎': 0.9997559785842896}
['急性阑尾炎']
['急性阑尾炎']
92
{'急性胆囊炎': 0.913597733806677}
{'急性胆囊炎': 0.9659513235092163}
['急性胆囊炎']
['急性胆囊炎']
93
{'上尿路结石': 0.9999994804319732}
{'上尿路结石': 0.9909017086029053}
['上尿路结石']
['上尿路结石']
94
{'上尿路结石': 0.9999999873962526}
{'上尿路结石': 0.9965270757675171}
['上尿路结石']
['上尿路结石']
95
{'急性胆管炎': 0.9999998239779972}
{'急性胆管炎': 0.9807661771774292}
['急性胆管炎']
['急性胆管炎']
96
{'上尿路结石': 0.9999996349339133}
{'上尿路结石': 0.9865378141403198}
['上尿路结石']
['上尿路结石']
97
{'上尿路结石': 0.9999999965819537}
{'上尿路结石': 0.9932702779769897}
['上尿路结石']
['上尿路结石']
98
{'上尿路结石': 0.9999999641471333}
{'上尿路结石': 0.9970980882644653}
['上尿路结石']
['上尿路结石']
99
{'急性胆囊炎': 0.9999971288566943}
{'急性胆囊炎': 0.9902770519256592, '急性胆管炎': 0.7288572788238525}
['急性胆囊炎']
['急性胆囊炎']
100
{'上尿路结石': 0.9999999741897593}
{'上尿路结石': 0.9886815547943115}

In [ ]:
### 人机比较结果：
### 基线：单诊断
Accuracy Score = 0.9006622516556292
F1 Score (Micro) = 0.9155844155844156
F1 Score (Macro) = 0.8993278401332572

#### 0.5
Accuracy Score = 0.9072847682119205
F1 Score (Micro) = 0.9190938511326862
F1 Score (Macro) = 0.9051479459533629

#### 0.5 nano （选择此模型）
Accuracy Score = 0.9072847682119205
F1 Score (Micro) = 0.9190938511326862
F1 Score (Macro) = 0.9051479459533629

#### 0.999
Accuracy Score = 0.8741721854304636
F1 Score (Micro) = 0.9292307692307693
F1 Score (Macro) = 0.9275322891646307

#### 0.999 nano
Accuracy Score = 0.8609271523178808
F1 Score (Micro) = 0.9141104294478527
F1 Score (Macro) = 0.9106413633621094

#### 0.7
Accuracy Score = 0.9072847682119205
F1 Score (Micro) = 0.923076923076923
F1 Score (Macro) = 0.9136326117392036

#### 0.9
Accuracy Score = 0.8874172185430463
F1 Score (Micro) = 0.9177215189873418
F1 Score (Macro) = 0.9113110671369573



In [ ]:
# 测试集比较结果：
# 基线：单诊断
Accuracy Score = 0.7990115321252059
F1 Score (Micro) = 0.8453922315308453
F1 Score (Macro) = 0.8192913387716643

# 0.5
Accuracy Score = 0.800658978583196
F1 Score (Micro) = 0.8462709284627092
F1 Score (Macro) = 0.8199911944715201

# 0.5 nano （选择此模型）
Accuracy Score = 0.800658978583196
F1 Score (Micro) = 0.8462709284627092
F1 Score (Macro) = 0.8199911944715201

#### 0.999
Accuracy Score = 0.7990115321252059
F1 Score (Micro) = 0.8602779809802488
F1 Score (Macro) = 0.836322123848968

# 0.999 nano
Accuracy Score = 0.7973640856672158
F1 Score (Micro) = 0.859882005899705
F1 Score (Macro) = 0.8397182595638456


# 0.7
Accuracy Score = 0.7990115321252059
F1 Score (Micro) = 0.8500376789751318
F1 Score (Macro) = 0.8221702109216127

# 0.9
Accuracy Score = 0.7973640856672158
F1 Score (Micro) = 0.8547904191616768
F1 Score (Macro) = 0.8281414487293165



In [ ]:
### 测试合成数据
# diag1
Accuracy Score = 0.7100494233937397
F1 Score (Micro) = 0.7242169595110771
F1 Score (Macro) = 0.6765752285887139

# diag3
Accuracy Score = 0.7742998352553542
F1 Score (Micro) = 0.821592649310873
F1 Score (Macro) = 0.7845213160447488

# diag1 + diag2
Accuracy Score = 0.7792421746293245
F1 Score (Micro) = 0.8132369722327882
F1 Score (Macro) = 0.7660079825633553

# diag1 + diag2 + diag3
Accuracy Score = 0.7813014827018122
F1 Score (Micro) = 0.8242588645611316
F1 Score (Macro) = 0.7874713870667375

# diag1 + diag2 + mdl
Accuracy Score = 0.7775947281713345
F1 Score (Micro) = 0.8204625637005095
F1 Score (Macro) = 0.781204559610193

# diag3 + mdl
Accuracy Score = 0.78665568369028
F1 Score (Micro) = 0.8310502283105023
F1 Score (Macro) = 0.7966473430255329

# diag1-3 + mdl
Accuracy Score = 0.7891268533772653
F1 Score (Micro) = 0.8409218662169758
F1 Score (Macro) = 0.8124648846526706



In [63]:
results2 = text_result_to_labels(single_pred(X_test))
calc_score(results2, y_test)

Accuracy Score = 0.7990115321252059
F1 Score (Micro) = 0.8453922315308453
F1 Score (Macro) = 0.8192913387716643


In [22]:
for idx, (r1, r2, y) in enumerate(zip(results1, results2, y_test)):
    if r1 != r2:
        print(idx, r1, r2, y, r1==y, r2==y)
    

7 [1, 0, 0, 0, 0, 0, 0, 1, 0] [0, 0, 0, 0, 0, 0, 0, 1, 0] [0, 0, 0, 0, 0, 0, 0, 1, 0] False True
8 [0, 1, 0, 0, 1, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 1, 0, 0, 0, 0] False False
17 [0, 1, 0, 0, 0, 1, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 1, 0, 0, 0] False False
27 [0, 0, 0, 0, 1, 0, 0, 0, 1] [0, 0, 0, 0, 1, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 1] False False
28 [0, 1, 0, 0, 1, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0] False True
32 [0, 1, 0, 0, 0, 1, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0] False True
33 [0, 0, 0, 0, 0, 0, 0, 0, 1] [0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 1] True False
36 [0, 1, 0, 0, 0, 1, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 1, 1, 0, 0, 0] False False
59 [0, 0, 0, 0, 0, 1, 1, 0, 0] [0, 0, 0, 0, 0, 1, 0, 0, 0] [0, 0, 0, 0, 0, 1, 0, 0, 0] False True
67 [0, 0, 0, 0, 1, 1, 0, 0, 0] [0, 0, 0, 0, 1, 0, 0, 0, 0] [0, 0, 0, 0, 1, 0, 0, 0, 0] False True
71 [0, 1, 0, 0, 1,

In [6]:
for idx, (X, y, ids) in enumerate(zip(X_test, y_test, ids_test)):
    print(idx + 1)
    
    # 单诊断
    result1 = {}
    pred = dg.predict_batch([X], pred_num=None, prob_delta=0.5, out_format='dict')[0]
    for r1 in pred:
        result1[r1['name']] = r1['prob']
    print(result1)
    
    # 多诊断
    text = '，'.join([str(x) for x in X])
#     for r2 in ie(text):
#         if schema in r2:
#             result2.extend(r2[schema])
    result2 = process_text(model, tokenizer, [text], prompt)[0]
    pprint(result2)

    result12 = merge_results(result1, result2)
    print(result12)
    
    # 金标准
    labels = []
    for idx, y_ in enumerate(y):
        if y_ == 1:
            labels.append(dieases[idx])
            
    print(labels)
    
    print('')

# # print(pred)
# for r, y in zip(pred, y_test):
#     s = ''
#     for r_ in r:
#         s_item = r_['name'] + '\t' + str(r_['prob'])
#         s = s + ',' + s_item if s != '' else s_item
#     print(dieases[y] + '\t' + s)
# # test_and_save(pred, y_test, ids_test, '人机_全特征')

1
{'肠梗阻': 0.9999999659165436}
{'肠梗阻': 0.9993325471878052}
['肠梗阻']
['肠梗阻']

2
{'消化道穿孔': 0.9039818226991675}
{'消化道穿孔': 0.9988742470741272}
['消化道穿孔']
['消化道穿孔']

3
{'上尿路结石': 0.9999990173352235}
{'上尿路结石': 0.9961837530136108}
['上尿路结石']
['上尿路结石']

4
{'肠梗阻': 0.9999987672145347}
{'肠梗阻': 0.996227502822876}
['肠梗阻']
['肠梗阻']

5
{'急性胰腺炎': 0.9999996776973803}
{'急性胆囊炎': 0.9681075811386108,
 '急性胆管炎': 0.7468574047088623,
 '急性胰腺炎': 0.9958971738815308}
['急性胰腺炎']
['急性胰腺炎']

6
{'急性胆管炎': 0.9999943033888808}
{'急性胆囊炎': 0.9890077710151672, '急性胆管炎': 0.7449440956115723}
['急性胆管炎']
['急性胆管炎']

7
{'上尿路结石': 0.9999900399410319}
{'上尿路结石': 0.9985754489898682}
['上尿路结石']
['上尿路结石']

8
{'卵巢囊肿': 0.5681908988861422}
{'急性阑尾炎': 0.9826538562774658}
['卵巢囊肿', '急性阑尾炎']
['卵巢囊肿']

9
{'急性胰腺炎': 0.9799711337775067}
{'急性胆囊炎': 0.6619626879692078,
 '急性胆管炎': 0.9819936752319336,
 '急性胰腺炎': 0.9957678318023682}
['急性胰腺炎', '急性胆管炎']
['急性胆管炎']

10
{'急性胰腺炎': 0.9999999995637072}
{'急性胰腺炎': 0.9991081357002258}
['急性胰腺炎']
['急性胰腺炎']

11
{'急性阑尾炎': 0.9951487

{'上尿路结石': 0.9987021684646606}
['上尿路结石']
['上尿路结石']

93
{'上尿路结石': 0.9999999873962526}
{'上尿路结石': 0.9976317286491394}
['上尿路结石']
['上尿路结石']

94
{'急性胆管炎': 0.9999998239779972}
{'急性胆管炎': 0.998458981513977}
['急性胆管炎']
['急性胆管炎']

95
{'上尿路结石': 0.9999996349339133}
{'上尿路结石': 0.9950928092002869}
['上尿路结石']
['上尿路结石']

96
{'上尿路结石': 0.9999999965819537}
{'上尿路结石': 0.9985950589179993}
['上尿路结石']
['上尿路结石']

97
{'上尿路结石': 0.9999999641471333}
{'上尿路结石': 0.9983420372009277}
['上尿路结石']
['上尿路结石']

98
{'急性胆囊炎': 0.9999971288566943}
{'急性胆囊炎': 0.9934564828872681}
['急性胆囊炎']
['急性胆囊炎']

99
{'上尿路结石': 0.9999999741897593}
{'上尿路结石': 0.9983702898025513}
['上尿路结石']
['上尿路结石']

100
{'上尿路结石': 0.9999938294821048}
{'上尿路结石': 0.9978406429290771}
['上尿路结石']
['上尿路结石']

101
{'上尿路结石': 0.999999998201579}
{'上尿路结石': 0.9985671639442444}
['上尿路结石']
['上尿路结石']

102
{'卵巢囊肿': 0.9972872014935007}
{'卵巢囊肿': 0.9681112170219421}
['卵巢囊肿']
['卵巢囊肿']

103
{'急性胆囊炎': 0.9999856011941085}
{'急性胆囊炎': 0.9963624477386475}
['急性胆囊炎']
['急性胆囊炎']

104
{'急性胰腺炎': 0.99796893796